# Who-Did-What Package Guide

Advanced Usage
Filtering by Subject or Object

python
Copy code
plot_svo_graph(svos, subject_filter='John')
Exporting Hypergraphs

python
Copy code
hypergraphs = export_hypergraphs(svos)
print(hypergraphs)


In [ ]:
plot_svo_graph
Visualizes the SVO relationships in a graph.

Parameters:

df (DataFrame): The DataFrame containing SVO data.
subject_filter (str): (Optional) Filter graph by a specific subject.
object_filter (str): (Optional) Filter graph by a specific object.
Example:

python
Copy code
plot_svo_graph(svos, subject_filter='Alice')
Other Functions
export_hypergraphs(df)
export_subj(df)
export_obj(df)
export_verb(df)


## Introduction

The Who-Did-What package provides tools for extracting Subject-Verb-Object (SVO) triples from textual data, resolving coreferences, and visualizing the extracted relationships as graphs. It's designed to assist in natural language processing tasks such as information extraction, text analysis, and knowledge graph construction.
Currently, the package only works in English.

<center><img src="https://raw.githubusercontent.com/RiccardoImprota/Who-did-What-Networks/refs/heads/main/WDW-usageexample.jpg" style="width:700px;height:393px;"></center>

---

## Installation

This package is currently not being featured on *pip* yet.

To install and use this package, follow these steps:

1. **Clone or Download the Repository**

   Download the package code and place it in your project directory or install it using `pip` if it's available on PyPI.

2. **Install Required Dependencies**

   The package relies on several Python libraries. Install them using `pip`:

   ```bash
   pip install spacy stanza networkx pandas matplotlib nltk fastcoref
Download Language Models

spaCy Model

Download the English language model for spaCy:

bash
Copy code
python -m spacy download en_core_web_sm
Stanza Model

Download the English models for stanza:

python
Copy code
import stanza
stanza.download('en')
NLTK Data

Download necessary data for NLTK:

python
Copy code
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
How the Package Works
The package processes text to extract SVO triples and builds a graph representing the relationships. Here's a high-level overview:

Text Preparation

Cleaning: Removes unnecessary characters and spaces.
Coreference Resolution: Replaces pronouns with the nouns they refer to using fastcoref or stanza.
SVO Extraction

Utilizes spaCy for dependency parsing to identify subjects, verbs, and objects.
Handles various grammatical structures, including compound subjects/objects and passive voice.
Graph Construction

Converts extracted SVO triples into a graph using networkx.
Nodes represent subjects, verbs, and objects.
Edges represent the relationships between them, including semantic relations like synonyms.
Visualization

Uses matplotlib to visualize the graph.
Nodes are color-coded based on sentiment (positive, negative, neutral).
Edges are styled to represent different types of relationships.
Usage Guide
1. Importing the Package
First, import the necessary functions:

python
Copy code
from your_package_name import extract_svos_from_text, plot_svo_graph
Replace your_package_name with the actual name of the package or the script file if you're running it locally.

2. Extracting SVOs from Text
Use the extract_svos_from_text function to process your text:

python
Copy code
text = "Alice went to the market. She bought apples."
svos_df = extract_svos_from_text(text)
Parameters:

text (str): The input text to process.
coref_solver (str, optional): The coreference resolution tool to use ('fastcoref' or 'stanza'). Default is 'fastcoref'.
Returns:

A pandas DataFrame containing the extracted SVO triples and additional metadata.
3. Visualizing the SVO Graph
Use the plot_svo_graph function to visualize the relationships:

python
Copy code
plot_svo_graph(svos_df)
Parameters:
df (DataFrame): The DataFrame returned by extract_svos_from_text.
subject_filter (str, optional): Filter the graph to focus on a specific subject.
object_filter (str, optional): Filter the graph to focus on a specific object.
4. Filtering the Graph
You can filter the visualization to focus on specific subjects or objects:

python
Copy code
# Filter by subject
plot_svo_graph(svos_df, subject_filter='Alice')

# Filter by object
plot_svo_graph(svos_df, object_filter='apples')
Examples
Example 1: Simple Sentence
python
Copy code
text = "The cat chased the mouse."
svos_df = extract_svos_from_text(text)
print(svos_df)
plot_svo_graph(svos_df)
Output:

A DataFrame displaying the SVO triples and a graph visualizing the relationship between the cat and the mouse.

Example 2: Text with Coreferences
python
Copy code
text = "John went to the store. He bought milk."
svos_df = extract_svos_from_text(text)
print(svos_df)
plot_svo_graph(svos_df)
Explanation:

Coreference resolution replaces "He" with "John".
The graph shows John as the subject performing both actions.
Example 3: Complex Sentences with Multiple SVOs
python
Copy code
text = """
Mary and her brother went to the park. They played soccer and enjoyed the sunny day.
The kids returned home when it got dark.
"""
svos_df = extract_svos_from_text(text)
print(svos_df)
plot_svo_graph(svos_df)
Explanation:

Handles compound subjects ("Mary and her brother") and compound verbs ("played soccer and enjoyed the sunny day").
Resolves "They" to "Mary and her brother".
Visualizes multiple SVO triples.
Additional Features
Coreference Resolution Options
You can choose between 'fastcoref' and 'stanza' for coreference resolution:

python
Copy code
# Using stanza
svos_df = extract_svos_from_text(text, coref_solver='stanza')
Valence Computation
The package computes the sentiment (positive, negative, neutral) of words, which influences node coloring in the graph.

Customization
You can modify visualization settings or extend the extraction functions to suit your needs.

Troubleshooting and Tips
Performance: Processing large texts may take time due to coreference resolution and parsing.
Dependencies: Ensure all models and data are properly downloaded to avoid errors.
Extending Functionality: The code is modular; you can integrate additional NLP tasks as needed.
Conclusion
This package offers a robust solution for extracting and visualizing SVO relationships in text, aiding in various NLP applications. Whether you're conducting text analysis or building knowledge graphs, this tool provides a solid foundation.

Note: Remember to replace placeholder names like your_package_name with the actual names used in your environment.

python
Copy code
# Sample Code Execution

# Importing the package
from your_package_name import extract_svos_from_text, plot_svo_graph

# Sample text
text = "Angela wrote a letter. She sent it to her friend."

# Extracting SVOs
svos_df = extract_svos_from_text(text)

# Displaying the DataFrame
print(svos_df)

# Plotting the graph
plot_svo_graph(svos_df)
Happy Coding!

Copy code





